# 🩺 Heart Disease Risk Classification — TFX Pipeline

Notebook ini merupakan implementasi pipeline TFX untuk memprediksi risiko penyakit jantung.  
Proyek ini meliputi tahapan: ingest data, analisis statistik, validasi skema, transformasi fitur, pelatihan model, hingga evaluasi dan deployment model.


In [1]:
import tensorflow as tf
from tfx.components import (
    CsvExampleGen,
    StatisticsGen,
    SchemaGen,
    ExampleValidator,
    Transform,
    Trainer,
    Tuner,
)
from tfx.proto import example_gen_pb2, trainer_pb2
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
import os

2025-10-13 00:42:51.759183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-13 00:42:51.991496: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-13 00:42:52.002231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.4/lib64
2025-10-13 00:42:52.002250: I tensorflow/stream_executor/cuda/cudart_stub.cc:29]

## 📊 Dataset

Dataset yang digunakan adalah **Heart Disease Dataset (UCI/Kaggle)** yang berisi data klinis pasien dari beberapa rumah sakit (Cleveland, Hungary, Switzerland, Long Beach V).  
Label `HeartDisease` bernilai:
- `1` → pasien memiliki indikasi penyakit jantung  
- `0` → tidak memiliki indikasi

Dataset ini cocok untuk masalah **klasifikasi biner** dengan 14 fitur utama seperti usia, tekanan darah, kolesterol, kadar gula, dan hasil EKG.

Nama: Heart Disease Dataset (Public Health Dataset, adaptasi UCI)

Ringkas: Data dari 4 rumah sakit (Cleveland, Hungary, Switzerland, Long Beach V). Umumnya dipakai 14 fitur subset. Label biner: 0 = tidak penyakit, 1 = penyakit.

Kolom yang kamu gunakan (sesuaikan dengan heart.csv kamu):

Numerik: Age, RestingBP, Cholesterol, FastingBS, MaxHR, Oldpeak

Kategorikal: Sex, ChestPainType, RestingECG, ExerciseAngina, ST_Slope

Label: HeartDisease

Alasan subset: fitur ini konsisten dengan variasi dataset modern Heart Disease, mudah untuk TFT (scaling + vocab), dan cukup kuat untuk baseline klasifikasi.

## ⚙️ Inisialisasi Pipeline TFX

Bagian ini menyiapkan struktur pipeline TFX, termasuk nama pipeline, direktori penyimpanan artifact, dan context interaktif.  
Pipeline TFX membantu mengatur setiap tahap pemrosesan data dan pelatihan model secara terstruktur dan reproducible.


In [2]:
PIPELINE_NAME = "arusdian-pipeline"
SCHEMA_PIPELINE_NAME = "arusdian-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join("pipelines", PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join("metadata", PIPELINE_NAME, "metadata.db")

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join("serving_model", PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## 📥 Data Ingestion (ExampleGen)

Komponen **ExampleGen** digunakan untuk membaca data mentah dari folder `data/` dan membaginya menjadi data **train** dan **eval**.  
Tahap ini memastikan data dapat diolah oleh komponen berikutnya dalam format TFRecord.


In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
            example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
        ]
    )
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## 📈 Analisis Statistik (StatisticsGen)

Komponen **StatisticsGen** menganalisis distribusi setiap fitur dalam dataset.  
Hasil statistik ini akan digunakan oleh komponen lain seperti SchemaGen dan ExampleValidator untuk mendeteksi anomali data.


In [7]:
statistics_gen = StatisticsGen(examples=example_gen.outputs["examples"])


interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

## 🧾 Pembuatan Skema (SchemaGen)

Komponen **SchemaGen** membuat skema data otomatis berdasarkan hasil analisis statistik.  
Skema ini berisi tipe data, rentang nilai, serta status fitur (misalnya: label, numerik, atau kategorikal).  
Skema membantu mendeteksi inkonsistensi data di tahap selanjutnya.


In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,required,,-
'ChestPainType',STRING,required,,'ChestPainType'
'Cholesterol',INT,required,,-
'ExerciseAngina',STRING,required,,'ExerciseAngina'
'FastingBS',INT,required,,-
'HeartDisease',INT,required,,-
'MaxHR',INT,required,,-
'Oldpeak',FLOAT,required,,-
'RestingBP',INT,required,,-


,Values
Domain,
'ChestPainType',"'ASY', 'ATA', 'NAP', 'TA'"
'ExerciseAngina',"'N', 'Y'"
'RestingECG',"'LVH', 'Normal', 'ST'"
'ST_Slope',"'Down', 'Flat', 'Up'"
'Sex',"'F', 'M'"


## 🚦 Validasi Data (ExampleValidator)

Komponen **ExampleValidator** menggunakan skema untuk memeriksa apakah ada data yang menyimpang dari ekspektasi.  
Contohnya:
- Nilai yang hilang,  
- Fitur dengan tipe tidak sesuai,  
- Distribusi yang menyimpang signifikan dari data pelatihan.

Tahap ini memastikan kualitas data tetap tinggi sebelum melanjutkan ke proses transformasi fitur.


In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"], schema=schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs["anomalies"])

## 🔄 Transformasi Data (Transform)

Komponen **Transform** menjalankan proses _feature engineering_ melalui modul `heart_transform.py`.  
Langkah ini memastikan semua fitur siap digunakan oleh model machine learning.

Transformasi yang dilakukan meliputi:
- **Normalisasi fitur numerik:** `Age`, `RestingBP`, `Cholesterol`, `MaxHR`, dll.  
- **Encoding fitur kategorikal:** `Sex`, `ChestPainType`, `RestingECG`, `ExerciseAngina`, `ST_Slope`.  
- Menentukan **kolom label:** `HeartDisease`.

Transformasi ini penting karena:
- Model NN sensitif terhadap skala input,  
- Fitur kategorikal harus diubah menjadi bentuk numerik,  
- Menjamin konsistensi antara data training dan serving.

Dengan preprocessing otomatis, pipeline dapat menghasilkan representasi data yang optimal untuk model.


In [ ]:
TRANSFORM_MODULE_FILE = f"{PIPELINE_ROOT}/modules/heart_transform.py"

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}
"""Transform module untuk TFX: normalisasi numerik & vokab kategorikal.

Fungsi utama:
- `preprocessing_fn(inputs)`: dipanggil oleh komponen TFX Transform untuk
  membangun graph preprocessing (TFT). Ia:
  * menormalisasi fitur numerik dengan z-score,
  * membuat & menerapkan vocabulary untuk fitur kategorikal (dengan 1 OOV),
  * meng-cast label ke int64,
  * dan memberi suffix `_xf` pada semua fitur keluaran.
"""

from typing import Dict, Mapping

import tensorflow as tf
import tensorflow_transform as tft

# ====== KONFIG ======
LABEL_KEY = "HeartDisease"

NUMERIC_FEATURE_KEYS = [
    "Age",
    "RestingBP",
    "Cholesterol",
    "FastingBS",
    "MaxHR",
    "Oldpeak",
]

CATEGORICAL_FEATURE_KEYS = [
    "Sex",
    "ChestPainType",
    "RestingECG",
    "ExerciseAngina",
    "ST_Slope",
]


def transformed_name(key: str) -> str:
    """Tambahkan sufiks '_xf' untuk menandai fitur pasca-transform."""
    return key + "_xf"


def preprocessing_fn(inputs: Mapping[str, tf.Tensor]) -> Dict[str, tf.Tensor]:
    """Mendefinisikan transformasi fitur untuk komponen TFX Transform.

    Args:
        inputs: Peta nama fitur mentah -> tf.Tensor (seperti pada schema raw).

    Returns:
        Dict nama fitur transform -> tf.Tensor siap dipakai model/trainer.
    """
    outputs: Dict[str, tf.Tensor] = {}

    # Fitur numerik: cast ke float32 lalu z-score
    for key in NUMERIC_FEATURE_KEYS:
        value = tf.cast(inputs[key], tf.float32)
        outputs[transformed_name(key)] = tft.scale_to_z_score(value)

    # Fitur kategorikal: buat & terapkan vocabulary (1 OOV bucket)
    for key in CATEGORICAL_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.compute_and_apply_vocabulary(
            inputs[key],
            num_oov_buckets=1,
            vocab_filename=key,
        )

    # Label sebagai int64
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs


Writing heart_transform.py


In [15]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE),
)
interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying heart_transform.py -> build/lib
installing to /tmp/tmpcbhv0q3y
running install
running install_lib
copying build/lib/heart_transform.py -> /tmp/tmpcbhv0q3y
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpcbhv0q3y/tfx_user_code_Transform-0.0+40926a625b36b1355e21b4ac37727afc7681c9f5f3fa5d3b156af6d408798680-py3.9.egg-info
running install_scripts
creating /tmp/tmpcbhv0q3y/tfx_user_code_Transform-0.0+40926a6

/home/ruvne/anaconda3/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/arusdian-pipeline/_wheels/tfx_user_code_Transform-0.0+40926a625b36b1355e21b4ac37727afc7681c9f5f3fa5d3b156af6d408798680-py3-none-any.whl
Processing ./pipelines/arusdian-pipeline/_wheels/tfx_user_code_Transform-0.0+40926a625b36b1355e21b4ac37727afc7681c9f5f3fa5d3b156af6d408798680-py3-none-any.whl
Processing ./pipelines/arusdian-pipeline/_wheels/tfx_user_code_Transform-0.0+40926a625b36b1355e21b4ac37727afc7681c9f5f3fa5d3b156af6d408798680-py3-none-any.whl
Instructions for updating:
Use ref() instead.


2025-10-13 00:43:04.412027: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-10-13 00:43:04.412306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.4/lib64
2025-10-13 00:43:04.412365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.4/lib64
2025-10-13 00:43:04.412398: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory

INFO:tensorflow:Assets written to: pipelines/arusdian-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/89eea1f59caa460fb1a744bdde8ab137/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## 🧠 Solusi Machine Learning (Tuner)

Model yang digunakan untuk menyelesaikan permasalahan ini adalah **Neural Network (NN)** menggunakan library **Keras (TensorFlow)**.  
Masalah yang dihadapi merupakan **klasifikasi biner** — apakah seorang pasien memiliki risiko penyakit jantung atau tidak — dengan fitur numerik dan kategorikal yang cukup kompleks.

### 🔍 Alasan Pemilihan Model:
1. **Kemampuan menangani hubungan non-linear**  
   Fitur klinis seperti usia, tekanan darah, dan kolesterol sering memiliki hubungan non-linear terhadap risiko penyakit jantung. Neural Network mampu memodelkan hubungan kompleks tersebut lebih baik daripada model linear seperti Logistic Regression.

2. **Fleksibilitas arsitektur**  
   NN dapat dengan mudah diatur jumlah layer dan neuron-nya agar sesuai dengan kompleksitas data. Ini membuatnya lebih adaptif terhadap dataset dengan fitur campuran (numerik & kategorikal).

3. **Kemampuan untuk dituning secara otomatis**  
   Melalui komponen **TFX Tuner**, pipeline dapat mencari kombinasi hyperparameter terbaik (misalnya jumlah neuron, _learning rate_, dan ukuran batch) untuk mencapai performa optimal secara sistematis.

4. **Efisiensi komputasi**  
   Dengan dataset berukuran sedang seperti ini, model NN dapat dilatih dengan cepat tanpa memerlukan sumber daya komputasi besar.

Secara keseluruhan, model Neural Network dipilih karena keseimbangan antara **akurasi tinggi, kemampuan generalisasi, dan kemudahan integrasi** ke dalam pipeline TFX yang end-to-end.


In [ ]:
TUNER_MODULE_FILE = f"{PIPELINE_ROOT}/modules/heart_tuner.py"

In [ ]:
%%writefile {TUNER_MODULE_FILE}
"""TFX Tuner module untuk model HeartDisease dengan KerasTuner BayesianOptimization.

Menyediakan:
- input_fn untuk membuat tf.data.Dataset dari artefak Transform
- _build_model(): membangun arsitektur model dengan hyperparameters
- tuner_fn(): entry-point yang mengembalikan objek tuner dan fit_kwargs
"""

from types import SimpleNamespace
from typing import Dict, Any

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tfx_bsl.public import tfxio
import keras_tuner as kt

# Konsisten dengan transform/trainer
LABEL_KEY = "HeartDisease"
NUMERIC_FEATURE_KEYS = [
    "Age",
    "RestingBP",
    "Cholesterol",
    "FastingBS",
    "MaxHR",
    "Oldpeak",
]
CATEGORICAL_FEATURE_KEYS = [
    "Sex",
    "ChestPainType",
    "RestingECG",
    "ExerciseAngina",
    "ST_Slope",
]


def transformed_name(key: str) -> str:
    """Tambahkan sufiks '_xf' pada nama fitur yang sudah di-transform."""
    return key + "_xf"


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size: int = 256):
    """Bangun tf.data.Dataset dari artefak Transform."""
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(
            batch_size=batch_size,
            label_key=transformed_name(LABEL_KEY),
        ),
        tf_transform_output.transformed_metadata.schema,
    )


def _vocab_size_from_tft(tf_transform_output, vocab_name: str) -> int:
    """Ambil ukuran vocabulary dari TFT (+1 untuk OOV)."""
    path = tf_transform_output.vocabulary_file_by_name(vocab_name)
    count = 0
    with tf.io.gfile.GFile(path, "r") as f:
        for _ in f:
            count += 1
    return count + 1  # OOV = 1


def _build_model(hp: kt.HyperParameters, tf_transform_output) -> keras.Model:
    """Bangun model tabular; minim locals agar lolos pylint R0914."""
    # ----- inputs -----
    inputs: Dict[str, keras.Input] = {}
    for key in NUMERIC_FEATURE_KEYS:
        name = transformed_name(key)
        inputs[name] = keras.Input(shape=(1,), dtype=tf.float32, name=name)

    cat_dims: Dict[str, int] = {}
    for key in CATEGORICAL_FEATURE_KEYS:
        name = transformed_name(key)
        inputs[name] = keras.Input(shape=(1,), dtype=tf.int32, name=name)
        cat_dims[key] = _vocab_size_from_tft(tf_transform_output, key)

    x_num = keras.layers.BatchNormalization(name="bn_numeric")(
        keras.layers.Concatenate(name="numeric_concat")(
            [inputs[transformed_name(k)] for k in NUMERIC_FEATURE_KEYS]
        )
    )

    cat_onehots = []
    for key in CATEGORICAL_FEATURE_KEYS:
        enc = keras.layers.CategoryEncoding(
            num_tokens=cat_dims[key],
            output_mode="one_hot",
            name=f"onehot_{key}",
        )(inputs[transformed_name(key)])
        enc = keras.layers.Reshape((cat_dims[key],))(enc)
        if cat_dims[key] > 1:
            enc = enc[:, 1:]  # drop-first
        cat_onehots.append(enc)

    x = x_num
    if cat_onehots:
        x = keras.layers.Concatenate(name="features_concat")(
            [x_num, keras.layers.Concatenate(name="cat_concat")(cat_onehots)]
        )

    # ----- hyperparameters (pakai dict untuk mengurangi variabel lokal) -----
    hp_vals: Dict[str, Any] = {
        "units1": hp.Int("units1", 32, 128, step=32),
        "units2": hp.Int("units2", 16, 64, step=16),
        "dropout": hp.Float("dropout", 0.2, 0.6, step=0.1),
        "l2": hp.Float("l2", 1e-5, 1e-3, sampling="log"),
        "lr": hp.Choice("lr", [1e-3, 5e-4, 3e-4]),
        "label_smoothing": hp.Float("label_smoothing", 0.0, 0.05, step=0.01),
    }

    x = keras.layers.Dense(
        hp_vals["units1"],
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(hp_vals["l2"]),
    )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(hp_vals["dropout"])(x)

    x = keras.layers.Dense(
        hp_vals["units2"],
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(hp_vals["l2"]),
    )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(hp_vals["dropout"])(x)

    outputs = keras.layers.Dense(1, activation="sigmoid", name="prob")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(hp_vals["lr"]),
        loss=keras.losses.BinaryCrossentropy(
            label_smoothing=hp_vals["label_smoothing"]
        ),
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),
            keras.metrics.AUC(name="auc"),
            keras.metrics.Precision(name="precision"),
            keras.metrics.Recall(name="recall"),
        ],
    )
    return model


def tuner_fn(fn_args):
    """Entry point TFX Tuner: siapkan BayesianOptimization dan argumen fit."""
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_ds = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=256,
    )
    eval_ds = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=256,
    )

    tuner = kt.BayesianOptimization(
        lambda hp: _build_model(hp, tf_transform_output),
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=25,
        num_initial_points=8,
        alpha=1e-4,
        beta=2.6,
        overwrite=True,
        directory=fn_args.working_dir,
        project_name="heart_tuning_bayes",
        seed=42,
    )

    fit_kwargs = {
        "x": train_ds,
        "validation_data": eval_ds,
        "steps_per_epoch": int(fn_args.train_steps),
        "validation_steps": int(fn_args.eval_steps),
        "epochs": 20,
        "callbacks": [
            tf.keras.callbacks.EarlyStopping(
                monitor="val_auc",
                mode="max",
                patience=3,
                restore_best_weights=True,
            )
        ],
        "verbose": 1,
    }

    return SimpleNamespace(tuner=tuner, fit_kwargs=fit_kwargs)



Writing heart_tuner.py


In [18]:
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=600),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=150),
)
tuner_result = interactive_context.run(tuner)

Trial 25 Complete [00h 00m 33s]
val_auc: 0.9256499409675598

Best val_auc So Far: 0.9328202605247498
Total elapsed time: 00h 13m 54s
Results summary
Results in pipelines/arusdian-pipeline/.temp/6/heart_tuning_bayes
Showing 10 best trials
Objective(name="val_auc", direction="max")

Trial 17 summary
Hyperparameters:
units1: 32
units2: 16
dropout: 0.4
l2: 0.0003652949862704224
lr: 0.0005
label_smoothing: 0.0
Score: 0.9328202605247498

Trial 07 summary
Hyperparameters:
units1: 96
units2: 16
dropout: 0.5
l2: 1.9584369813174676e-05
lr: 0.0005
label_smoothing: 0.0
Score: 0.9309388995170593

Trial 16 summary
Hyperparameters:
units1: 64
units2: 16
dropout: 0.5
l2: 5.9303593966932596e-05
lr: 0.0005
label_smoothing: 0.02
Score: 0.9298495650291443

Trial 04 summary
Hyperparameters:
units1: 32
units2: 16
dropout: 0.4
l2: 0.00023360487671257027
lr: 0.0003
label_smoothing: 0.01
Score: 0.928219735622406

Trial 10 summary
Hyperparameters:
units1: 32
units2: 16
dropout: 0.4
l2: 0.0005145089528400782
lr:

In [19]:
import os, json, glob, time
from pathlib import Path
import pandas as pd

best_hp_uri = None
if "tuner_result" in globals():
    try:
        best_hp_uri = tuner_result.artifact_dict["best_hyperparameters"][0].uri
        print("[INFO] Ambil dari tuner_result:", best_hp_uri)
    except Exception as e:
        print("[WARN] Gagal ambil dari tuner_result:", e)

if best_hp_uri is None:
    patterns = [
        os.path.join(
            PIPELINE_ROOT, "Tuner", "**", "best_hyperparameters", "**", "*.json"
        ),
        os.path.join(
            PIPELINE_ROOT, "Tuner", "**", "best_hyperparameters", "**", "*.txt"
        ),
        os.path.join(PIPELINE_ROOT, "Tuner", "**", "best_hyperparameters", "**", "*"),
    ]
    candidates = []
    for pat in patterns:
        candidates.extend(glob.glob(pat, recursive=True))
    candidates = [p for p in candidates if os.path.isfile(p)]
    if not candidates:
        raise FileNotFoundError(
            "Tidak menemukan artefak best_hyperparameters. "
            "Jika baru saja menjalankan Tuner, simpan hasilnya: tuner_result = interactive_context.run(tuner)"
        )
    candidates.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    sel = candidates[0]
    best_hp_uri = os.path.dirname(sel)
    print("[INFO] Ambil dari filesystem:", sel)
else:
    files = glob.glob(os.path.join(best_hp_uri, "*.json")) + glob.glob(
        os.path.join(best_hp_uri, "*.txt")
    )
    if files:
        sel = sorted(files, key=os.path.getmtime, reverse=True)[0]
    else:
        # fallback: ambil file apa saja
        any_files = glob.glob(os.path.join(best_hp_uri, "*"))
        sel = (
            sorted(any_files, key=os.path.getmtime, reverse=True)[0]
            if any_files
            else None
        )

if sel is None:
    raise FileNotFoundError(f"Tidak ada file di {best_hp_uri}")

raw = open(sel, "r").read()
values = {}
try:
    cfg = json.loads(raw)
    # format umum KerasTuner
    if isinstance(cfg, dict):
        if "hyperparameters" in cfg and isinstance(cfg["hyperparameters"], dict):
            values = cfg["hyperparameters"].get("values", {}) or {}
        if not values and "values" in cfg and isinstance(cfg["values"], dict):
            values = cfg["values"]
        if not values and "space" in cfg and isinstance(cfg["space"], list):
            for item in cfg["space"]:
                n = item.get("name")
                v = item.get("value", item.get("default"))
                if n is not None and v is not None:
                    values[n] = v
except Exception:
    pass

if not values:
    for line in raw.splitlines():
        if ":" in line and len(line.split(":")[0].strip()) > 0:
            k, v = line.split(":", 1)
            values[k.strip()] = v.strip()

df_hp = pd.DataFrame(sorted(values.items()), columns=["hyperparameter", "value"])
display(df_hp)

ART_DIR = Path("artifacts")
ART_DIR.mkdir(exist_ok=True)
best_json_path = ART_DIR / "best_hparams.json"
best_md_path = ART_DIR / "best_hparams.md"

with open(best_json_path, "w") as f:
    json.dump(values, f, indent=2)

with open(best_md_path, "w") as f:
    f.write("### Best Hyperparameters (Tuner)\n\n")
    for k, v in sorted(values.items()):
        f.write(f"- **{k}**: {v}\n")

print(f"✅ Tersimpan: {best_json_path}")
print(f"✅ Tersimpan: {best_md_path}")

[WARN] Gagal ambil dari tuner_result: 'ExecutionResult' object has no attribute 'artifact_dict'
[INFO] Ambil dari filesystem: pipelines/arusdian-pipeline/Tuner/best_hyperparameters/6/best_hyperparameters.txt


,hyperparameter,value
0,dropout,0.400000
1,l2,0.000365
2,label_smoothing,0.000000
3,lr,0.000500
4,units1,32.000000
5,units2,16.000000


✅ Tersimpan: artifacts/best_hparams.json
✅ Tersimpan: artifacts/best_hparams.md


## 🏋️‍♂️ Pelatihan Model (Trainer)

Komponen **Trainer** berfungsi melatih model dengan konfigurasi terbaik yang telah ditemukan oleh Tuner.  
Model Neural Network yang digunakan memiliki beberapa layer tersembunyi (_hidden layers_) dengan aktivasi **ReLU**, serta **sigmoid** pada output layer untuk prediksi probabilitas biner.

Struktur umum model:
- Input layer disesuaikan dengan fitur hasil transformasi.  
- 1–2 hidden layers (dengan jumlah neuron optimal hasil tuning).  
- Output layer (1 neuron, aktivasi sigmoid).

Model dikompilasi menggunakan:
- **Loss:** `binary_crossentropy` (karena klasifikasi biner)  
- **Optimizer:** `adam` (fleksibel dan efisien)  
- **Metrics:** `accuracy`, `AUC`, `precision`, dan `recall`

Proses pelatihan dilakukan pada data hasil transformasi dengan rasio train–eval sekitar 80:20.  
Hasil model akan disimpan sebagai artifact untuk digunakan oleh komponen Evaluator dan Pusher pada tahap berikutnya.

Model ini diharapkan mampu memberikan **prediksi yang akurat dan stabil**, serta mudah untuk di-deploy karena mengikuti format standar TensorFlow/TFX.


In [ ]:
TRAINER_MODULE_FILE = f"{PIPELINE_ROOT}/modules/heart_trainer.py"

In [ ]:
%%writefile {TRAINER_MODULE_FILE}
"""Trainer TFX untuk model klasifikasi penyakit jantung (HeartDisease).

Berisi:
- utilitas input_fn dari artefak Transform
- perakitan model tabular (numeric + categorical one-hot)
- pembacaan hyperparameter (opsional) dari Tuner
- entrypoint `run_fn` untuk komponen TFX Trainer
"""

import json
from typing import Any, Dict, Optional

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow import keras
from tfx_bsl.public import tfxio

# --- konsisten dgn transform_module.py ---
LABEL_KEY = "HeartDisease"
NUMERIC_FEATURE_KEYS = [
    "Age",
    "RestingBP",
    "Cholesterol",
    "FastingBS",
    "MaxHR",
    "Oldpeak",
]
CATEGORICAL_FEATURE_KEYS = [
    "Sex",
    "ChestPainType",
    "RestingECG",
    "ExerciseAngina",
    "ST_Slope",
]


def transformed_name(key: str) -> str:
    """Tambahkan sufiks '_xf' untuk nama fitur yang sudah di-transform."""
    return key + "_xf"


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size: int = 256):
    """Bangun tf.data.Dataset dari artefak Transform."""
    return data_accessor.tf_dataset_factory(
        file_pattern,
        tfxio.TensorFlowDatasetOptions(
            batch_size=batch_size,
            label_key=transformed_name(LABEL_KEY),
        ),
        tf_transform_output.transformed_metadata.schema,
    )


def _vocab_size_from_tft(tf_transform_output, vocab_name: str) -> int:
    """Ambil ukuran vocabulary dari TFT (+1 untuk OOV)."""
    path = tf_transform_output.vocabulary_file_by_name(vocab_name)
    n = 0
    with tf.io.gfile.GFile(path, "r") as f:
        for _ in f:
            n += 1
    return n + 1  # OOV=1 di Transform


def _get_hparam(hparams: Any, name: str, default: Any) -> Any:
    """Ambil hparam dari Tuner bila ada; selain itu pakai default.

    Mendukung:
      - dict JSON
      - keras_tuner.HyperParameters (punya method get)
    """
    if hparams is None:
        return default

    if isinstance(hparams, dict):
        return hparams.get(name, default)

    # Generic: objek yang punya method .get(name)
    try:
        return hparams.get(name)  # type: ignore[attr-defined]
    except (AttributeError, TypeError):
        return default


def _build_model(tf_transform_output, hparams: Optional[Dict[str, Any]] = None):
    """Bangun model tabular dengan one-hot drop-first + regularisasi; minim locals."""
    # Kumpulkan semua hyperparameter ke satu dict supaya tidak banyak variabel lokal.
    hp = {
        "units1": int(_get_hparam(hparams, "units1", 64)),
        "units2": int(_get_hparam(hparams, "units2", 32)),
        "dropout": float(_get_hparam(hparams, "dropout", 0.5)),
        "l2": float(_get_hparam(hparams, "l2", 1e-4)),
        "lr": float(_get_hparam(hparams, "lr", 5e-4)),
        "label_smoothing": float(_get_hparam(hparams, "label_smoothing", 0.03)),
    }

    # ---------- inputs ----------
    inputs: Dict[str, keras.Input] = {}
    for k in NUMERIC_FEATURE_KEYS:
        inputs[transformed_name(k)] = keras.Input(
            shape=(1,), dtype=tf.float32, name=transformed_name(k)
        )

    cat_dims: Dict[str, int] = {}
    for k in CATEGORICAL_FEATURE_KEYS:
        inputs[transformed_name(k)] = keras.Input(
            shape=(1,), dtype=tf.int32, name=transformed_name(k)
        )
        cat_dims[k] = _vocab_size_from_tft(tf_transform_output, k)

    # ---------- numeric pipeline ----------
    x_num = keras.layers.BatchNormalization(name="bn_numeric")(
        keras.layers.Concatenate(name="numeric_concat")(
            [inputs[transformed_name(k)] for k in NUMERIC_FEATURE_KEYS]
        )
    )

    # ---------- categorical pipeline (one-hot + drop-first) ----------
    cat_onehots = []
    for k in CATEGORICAL_FEATURE_KEYS:
        enc = keras.layers.CategoryEncoding(
            num_tokens=cat_dims[k], output_mode="one_hot", name=f"onehot_{k}"
        )(inputs[transformed_name(k)])
        enc = keras.layers.Reshape((cat_dims[k],))(enc)
        if cat_dims[k] > 1:
            enc = enc[:, 1:]  # drop-first
        cat_onehots.append(enc)

    x = x_num
    if cat_onehots:
        x = keras.layers.Concatenate(name="features_concat")(
            [x_num, keras.layers.Concatenate(name="cat_concat")(cat_onehots)]
        )

    # ---------- MLP + regularisasi ----------
    x = keras.layers.Dense(
        hp["units1"],
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(hp["l2"]),
    )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(hp["dropout"])(x)

    x = keras.layers.Dense(
        hp["units2"],
        activation="relu",
        kernel_regularizer=keras.regularizers.l2(hp["l2"]),
    )(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(hp["dropout"])(x)

    model = keras.Model(
        inputs=inputs,
        outputs=keras.layers.Dense(1, activation="sigmoid", name="prob")(x),
    )
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp["lr"]),
        loss=keras.losses.BinaryCrossentropy(label_smoothing=hp["label_smoothing"]),
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),
            keras.metrics.AUC(name="auc"),
            keras.metrics.Precision(name="precision"),
            keras.metrics.Recall(name="recall"),
        ],
    )
    return model


# pylint: disable=abstract-method
class ServingModule(tf.Module):
    """Bungkus model & TFT layer agar resources (vocab tables) ter-track saat export."""

    def __init__(
        self,
        model: tf.keras.Model,
        tf_transform_output: tft.TFTransformOutput,
        label_key: str,
    ):
        super().__init__()
        self.model = model
        # tracked
        self.tft_layer = tf_transform_output.transform_features_layer()
        spec = tf_transform_output.raw_feature_spec()
        # hapus label dari spesifikasi serving
        spec.pop(label_key, None)
        self.raw_feature_spec = spec

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")]
    )
    def serve(self, serialized_tf_examples):
        """Signature serving_default: menerima tf.Example ter-serialize."""
        raw_features = tf.io.parse_example(
            serialized_tf_examples, self.raw_feature_spec
        )
        transformed = self.tft_layer(raw_features)
        probs = self.model(transformed)
        return {"outputs": probs}


def _parse_hparams(fn_args) -> Optional[Dict[str, Any]]:
    """Parse hyperparameters dari fn_args; kembalikan dict bila ada."""
    if not getattr(fn_args, "hyperparameters", None):
        return None

    cfg = fn_args.hyperparameters
    if isinstance(cfg, dict):
        return cfg

    # Jika datang sebagai JSON string
    try:
        return json.loads(cfg)
    except (json.JSONDecodeError, TypeError):
        return None


def _make_callbacks():
    """Buat daftar callback pelatihan."""
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_auc",
        mode="max",
        factor=0.5,
        patience=2,
        verbose=1,
    )
    early_stop = keras.callbacks.EarlyStopping(
        monitor="val_auc",
        mode="max",
        patience=5,
        restore_best_weights=True,
        verbose=1,
    )
    return [reduce_lr, early_stop]


def run_fn(fn_args):
    """Entry point untuk komponen TFX Trainer."""
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # --- baca hyperparameters dari Tuner (jika ada) ---
    hparams = _parse_hparams(fn_args)

    # datasets
    train_ds = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=256,
    )
    eval_ds = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        batch_size=256,
    )

    # build model (parametris)
    model = _build_model(tf_transform_output, hparams=hparams)

    callbacks = _make_callbacks()

    # steps (hindari infinite dataset)
    steps_per_epoch = (
        int(fn_args.train_steps) if getattr(fn_args, "train_steps", None) else None
    )
    validation_steps = (
        int(fn_args.eval_steps) if getattr(fn_args, "eval_steps", None) else None
    )
    if steps_per_epoch is None or validation_steps is None:
        raise ValueError("Trainer membutuhkan num_steps pada TrainArgs dan EvalArgs.")

    model.fit(
        train_ds,
        validation_data=eval_ds,
        epochs=40,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1,
    )

    # export SavedModel dengan signature serving_default (raw tf.Example)
    module = ServingModule(model, tf_transform_output, LABEL_KEY)
    tf.saved_model.save(
        module,
        fn_args.serving_model_dir,
        signatures={"serving_default": module.serve},
    )


Writing heart_trainer.py


In [22]:

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=200),
)
interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying heart_trainer.py -> build/lib
copying heart_transform.py -> build/lib
copying heart_tuner.py -> build/lib
installing to /tmp/tmpit3fvkhj
running install
running install_lib
copying build/lib/heart_trainer.py -> /tmp/tmpit3fvkhj
copying build/lib/heart_tuner.py -> /tmp/tmpit3fvkhj
copying build/lib/heart_transform.py -> /tmp/tmpit3fvkhj
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpit3fvkhj/tfx_user_code_Trainer-0.0+a

/home/ruvne/anaconda3/envs/mlops-tfx/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


Processing ./pipelines/arusdian-pipeline/_wheels/tfx_user_code_Trainer-0.0+aa7e030de14e70f47f27742ded469385eeeb423cc606f68159efc353fe74464e-py3-none-any.whl
Epoch 1/40
800/800 [==============================] - 11s 12ms/step - loss: 0.4186 - accuracy: 0.8387 - auc: 0.9110 - precision: 0.8536 - recall: 0.8519 - val_loss: 0.3967 - val_accuracy: 0.8477 - val_auc: 0.9259 - val_precision: 0.8794 - val_recall: 0.8500 - lr: 5.0000e-04
Epoch 2/40
800/800 [==============================] - 8s 10ms/step - loss: 0.3109 - accuracy: 0.8960 - auc: 0.9572 - precision: 0.8980 - recall: 0.9140 - val_loss: 0.4165 - val_accuracy: 0.8381 - val_auc: 0.9208 - val_precision: 0.8644 - val_recall: 0.8500 - lr: 5.0000e-04
Epoch 3/40
798/800 [============================>.] - ETA: 0s - loss: 0.2808 - accuracy: 0.9118 - auc: 0.9679 - precision: 0.9135 - recall: 0.9269
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
800/800 [==============================] - 8s 10ms/step - loss: 0.2807 

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines/arusdian-pipeline/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/arusdian-pipeline/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## 🧩 Model Resolver

Komponen **Resolver** bertugas memilih model terbaik yang sudah ada sebelumnya (baseline) untuk dibandingkan dengan model baru hasil pelatihan saat ini.  
Proses ini penting untuk menjaga **konsistensi performa model dari waktu ke waktu**.

Jika model baru memiliki performa yang lebih baik berdasarkan metrik evaluasi (misalnya accuracy atau AUC lebih tinggi), maka model tersebut akan dipilih sebagai model aktif untuk di-*deploy*.  
Sebaliknya, jika model baru tidak lebih baik, pipeline dapat mencegah deployment otomatis agar tidak menurunkan performa sistem yang sedang berjalan.

Dengan adanya Resolver, pipeline TFX dapat menjalankan proses _continuous training_ dengan jaminan bahwa hanya model dengan performa terbaik yang digunakan di lingkungan produksi.


In [23]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy,
)
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## 📊 Evaluasi Model (Evaluator)

Komponen **Evaluator** digunakan untuk menilai performa model hasil pelatihan berdasarkan metrik evaluasi yang telah ditentukan.  
Evaluator secara otomatis membandingkan hasil model terhadap data evaluasi, dan juga dapat dibandingkan dengan model sebelumnya (baseline).

Metrik yang digunakan dalam proyek ini antara lain:
- **Accuracy** – mengukur seberapa sering prediksi model benar.  
- **Precision** – menunjukkan proporsi prediksi positif yang benar.  
- **Recall** – mengukur kemampuan model mendeteksi kasus positif dengan benar.  
- **AUC (Area Under Curve)** – menilai kemampuan model membedakan kelas positif dan negatif secara keseluruhan.

Langkah evaluasi ini membantu memastikan bahwa model tidak hanya akurat di data pelatihan, tetapi juga mampu melakukan generalisasi dengan baik pada data baru.


In [24]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key="HeartDisease")],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(class_name="ExampleCount"),
                tfma.MetricConfig(class_name="AUC"),
                tfma.MetricConfig(class_name="FalsePositives"),
                tfma.MetricConfig(class_name="TruePositives"),
                tfma.MetricConfig(class_name="FalseNegatives"),
                tfma.MetricConfig(class_name="TrueNegatives"),
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.5}
                        ),
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": 0.0001},
                        ),
                    ),
                ),
            ]
        )
    ],
)

In [25]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config,
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [26]:
eval_result = evaluator.outputs["evaluation"].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(tfma_result)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'example_cou…

## 🚀 Deployment Model (Pusher)

Komponen **Pusher** akan mengirimkan (_push_) model terbaik ke direktori **serving** setelah Evaluator dan Resolver menyetujui performanya.  
Langkah ini merupakan tahap akhir dari pipeline TFX, di mana model siap digunakan untuk melakukan prediksi nyata.

Pada tahap ini:
- Model disimpan dalam format **SavedModel** TensorFlow.  
- Model dapat digunakan oleh aplikasi atau API untuk melakukan inferensi (prediksi risiko penyakit jantung).  
- Pipeline memastikan hanya model dengan performa terbaik yang didorong ke tahap serving.

Tahapan ini menjadikan pipeline bersifat **end-to-end dan production-ready**, karena mengotomatisasi seluruh proses mulai dari data mentah hingga deployment model.


In [27]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR
        )
    ),
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## 🏁 Kesimpulan

Pipeline TFX untuk **Heart Disease Risk Classification** berhasil dibangun secara lengkap, mencakup seluruh tahapan machine learning pipeline.  
Model Neural Network yang digunakan menunjukkan performa baik dalam memprediksi risiko penyakit jantung dan mudah dioptimalkan menggunakan Tuner.

Komponen **Evaluator**, **Resolver**, dan **Pusher** memastikan pipeline hanya memproduksi dan mendistribusikan model dengan performa terbaik.  
Dengan demikian, sistem ini siap digunakan untuk implementasi nyata dan pembaruan model otomatis di masa mendatang.
